In [1]:
import tensorflow as tf
import os

### 什么是TFRecords文件

TFRecords其实是一种二进制文件，虽然他不如其他格式好理解，但是它能更好的利用内存，更方便复制和移动，并且不需要单独的标签文件。TFRecords文件包含了`tf.train.Example`协议内存块（protocol buffer）（协议内存块包含了字段Feature）。可以获取你的数据，将数据填入到`Example`协议内存块（protocol buffer），将协议内存块序列化一个字符串，并且通过`tf.python_io.TFRecordWriter`写入到TFRecords文件。

* 文件格式：`*.tfrecords`

* 他能更好地利用内存，更方便复制和移动

* Example：方便存储更多的图片的信息，特征值、目标值、通道等等，不需要更多去处理读取出来的结果


### Example结构解析

`tf.train.Example`协议内存块(protocol buffer)(协议内存块包含了字段 Features)，Features包含了一个Feature字段，Features中包含要写入的数据、并指明数据类型。这是一个样本的结构，批数据需要循环存入这样的结构

 `example = tf.train.Example(features=tf.train.Features(feature={
                "image": tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
                "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
            }))`
            
            
`tf.train.Example(features=None)`写入tfrecords文件

    `features`:tf.train.Features类型的特征实例

    `return`：example格式协议块

`tf.train.Features(feature=None)`构建每个样本的信息键值对

    `feature`:字典数据,key为要保存的名字

    `value`为tf.train.Feature实例

    `return`:Features类型

`tf.train.Feature(options)`
    options：例如
    `
    bytes_list=tf.train. BytesList(value=[Bytes])
    int64_list=tf.train. Int64List(value=[Value])`
    支持存入的类型如下
    `
    tf.train.Int64List(value=[Value])
    tf.train.BytesList(value=[Bytes])
    tf.train.FloatList(value=[value])`

这种结构是不是很好的解决了数据和标签(训练的类别标签)或者其他属性数据存储在同一个文件中

### 分析

* 构造存储实例，`tf.python_io.TFRecordWriter(path)`

    * 写入`tfrecords`文件
    
    * `path`：TFRecords文件的路径
    
    * `return`：写文件
    
    * method:
        * `write(record)`：向文件中写入一个example
        * `close()`：关闭文件写入器
        
* 循环将数据填入到`Example`协议内存块(protocol buffer)

In [15]:
class CifarRead(object):
    """读取CIFRA10类别的二进制文件"""
    def __init__(self):
        # 每个样本的图片属性
        self.height = 32
        self.width = 32
        self.channel = 3
        
        # bytes
        self.label_bytes = 1
        # 3072
        self.image_bytes = self.height * self.height * self.channel
        # 3073
        self.all_bytes = self.label_bytes + self.image_bytes
       
    def bytes_read(self, file_list):
        """读取二进制，解码为张量"""
        # 1.构造文件队列
        file_queue = tf.train.string_input_producer(file_list)

        # 2.读取二进制文件
        #   默认必须制定读取一个样本
        reader = tf.FixedLengthRecordReader(self.all_bytes)
        _, value = reader.read(file_queue)
        
        # 3.解码操作
        # （？, ） --> (3073, ) = label(1, ) + feature(3072, )
        label_img = tf.decode_raw(value, tf.uint8)
        print(label_img)
        
        # 为了训练方便，要把特征值和目标值分开处理，使用tf.slice
        label = tf.cast(tf.slice(label_img, [0], [self.label_bytes]), tf.int32)
        
        image = tf.slice(label_img, [self.label_bytes], [self.image_bytes])
        print(label, image)
        
        # 处理类型和图片数据形状，图片形状[32, 32, 3]
        # reshape(3071, ) ---> [channel, height, width]
        # 接下来使用tf.trainspose, 0, 1, 2 分别表示三个维度 
        # transpose[channel, height, width] ---> [height, width, channel]
        depth_major = tf.reshape(image, [self.channel, self.height, self.width])
        image_reshap = tf.transpose(depth_major, [1, 2, 0])
        print(depth_major)
        
        # 4.批处理操作
        img_batch, label_batch = tf.train.batch([image_reshap, label], batch_size=10, num_threads=1, capacity=10)
        
        return img_batch, label_batch
    
    def write_to_tfrecords(self, img_batch, label_batch):
        """将数据写入TFRecords文件"""
        # 构造一个TFRecords的存储器
        writer = tf.python_io.TFRecordWriter("C:\\Users\\Administrator\\Git\\CFturb\\Deep_Learning\\cifar.tfrecords")
        
        # 循环将每个样本构造成一个example，然后序列化写入
        for i in range(10):
            
            # 取出相应的第i个样本的特征值和目标值
            # 写入的是具体的张量的值，不是op的名字，然后转换成byteslist的形式
            image = img_batch[i].eval().tostring()
            
            # 把label变成整型
            label = label_batch[i].eval()[0]
                        
            # 每个样本的example
            example = tf.train.Example(features=tf.train.Features(feature={
                "image": tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
                "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
            }))
        
            # 写入第i个样本的example
            writer.write(example.SerializeToString())
            
        writer.close()        
        
        return None
    
    def read_tfrecords(self):
        """读取TFRecords文件"""
        # 构造文件队列
        file_queue = tf.train.string_input_producer(["C:\\Users\\Administrator\\Git\\CFturb\D\eep_Learning\\cifar.tfrecords"])
        
        # tf.TFRecordReader 读取TFRecords数据并进行解析example协议
        reader = tf.TFRecordReadr()
        
        # 默认只读取一个样本
        _, value = reader.read(file_queue)
        
        # 解析example协议
        feature = tf.parse_single_example(value, features={"image": tf.FixedLenFeature([], tf.string),
                                                           "label": tf.FixedLenFeature([], tf.int64)})

        # 二进制的形式必须解码
        image = tf.decode_raw(feature['image'], tf.int8)
        label = tf.cast(feature['label'], tf.int32) 
        
        # 形状、类型 [32, 32, 3] --> bytes --> tf.uint8
        image_reshape = tf.reshape(image, [self.heiget, self.width, self.channel])
        
        # 批处理
        image_batch, label_batch = tf.train.batch([image_reshape, label], batch_size=10, num_threads=1, capacity=1)

        return img_batch, label_batch



In [16]:
if __name__ == '__main__':
    filename = os.listdir("E:\\AI\\data\\cifar10\\cifar-10-batches-bin\\")
    file_list = [os.path.join("E:\\AI\\data\\cifar10\\cifar-10-batches-bin\\", file) for file in filename if file[-3 : ] == "bin"]
#     print(file_list)
    
    # 实例化类
    cr = CifarRead()
        
    # img_batch, label_batch = cr.bytes_read(file_list)

    # 读取TFRecords的结构
    image_batch, label_batch = cr.read_tfrecords()

    with tf.Session() as sess:
        
        # 创建线程回收的协调员
        coord = tf.train.Coordinator()
        
        # 需要手动开启子线程去进行批处理读取数据到队列的操作
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        
        print(sess.run([img_batch, label_batch]))
        
        # 写入文件
        cr.write_to_tfrecords(img_batch, label_batch)
        
        # 回收线程
        coord.request_stop()
        
        coord.join(threads)


### 读取TFRecords文件

读取这种文件整个过程与其他文件一样，只不过需要有个解析Example的步骤。从TFRecords文件中读取数据， 可以使用tf.TFRecordReader的tf.parse_single_example解析器。这个操作可以将Example协议内存块(protocol buffer)解析为张量。 

多了解析example的一个步骤

        feature = tf.parse_single_example(values, features={
            "image": tf.FixedLenFeature([], tf.string),
            "label": tf.FixedLenFeature([], tf.int64)
        })
        
        
`tf.parse_single_example(serialized,features=None,name=None)`

解析一个单一的Example原型

serialized：标量字符串Tensor，一个序列化的Example

features：dict字典数据，键为读取的名字，值为FixedLenFeature

return:一个键值对组成的字典，键为读取的名字

tf.FixedLenFeature(shape,dtype)

shape：输入数据的形状，一般不指定,为空列表

dtype：输入数据类型，与存储进文件的类型要一致

类型只能是float32,int64,string